In [1]:
import re
import pandas as pd

def parse_verification_report(path):
    """
    Lit un fichier texte contenant un rapport au format donné
    et extrait les informations importantes dans un dictionnaire.
    """

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    data = {}

    # Final verified acc
    m = re.search(r"Final verified acc:\s*([\d\.]+)%", text)
    if m:
        data["final_verified_accuracy"] = float(m.group(1))

    # Problem instances count
    m = re.search(r"Problem instances count:\s*(\d+)", text)
    if m:
        data["problem_instances_count"] = int(m.group(1))

    # total verified, total falsified, timeout
    m = re.search(
        r"total verified.*?:\s*(\d+)\s*,\s*total falsified.*?:\s*(\d+)\s*,\s*timeout:\s*(\d+)",
        text
    )
    if m:
        data["verified"] = int(m.group(1))
        data["falsified"] = int(m.group(2))
        data["timeout"] = int(m.group(3))

    # mean / max time for all instances
    m = re.search(
        r"mean time for ALL instances.*?:([\d\.]+).*?max time:\s*([\d\.]+)",
        text
    )
    if m:
        data["mean_time_all"] = float(m.group(1))
        data["max_time_all"] = float(m.group(2))

    # mean / max time for verified SAFE
    m = re.search(
        r"mean time for verified SAFE instances.*?:\s*([\d\.]+).*?max time:\s*([\d\.]+)",
        text
    )
    if m:
        data["mean_time_safe"] = float(m.group(1))
        data["max_time_safe"] = float(m.group(2))

    # safe-incomplete list
    m = re.search(r"safe-incomplete.*?index:\s*\[(.*?)\]", text)
    if m:
        indices = m.group(1).strip()
        data["safe_incomplete_indices"] = (
            [] if indices == "" else [int(x) for x in indices.split(",")]
        )

    return data


In [2]:
result = parse_verification_report("/share/homes/boyerma/FastSDPCertification/results/benchmark/6x100-0.026/Branch-and-Bound-beta-CROWN/data_index=0-71.txt")
print(result)


{'final_verified_accuracy': 97.22222222222221, 'problem_instances_count': 72, 'verified': 70, 'falsified': 2, 'timeout': 0, 'mean_time_all': 0.5151906059561167, 'max_time_all': 8.080828666687012, 'mean_time_safe': 0.5295758826392037, 'max_time_safe': 8.080828666687012, 'safe_incomplete_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71]}


In [3]:
import os
import sys

folder = "/share/homes/boyerma/FastSDPCertification/results/benchmark/6x100-0.026/Branch-and-Bound-beta-CROWN/" 
results = pd.DataFrame(columns=[
    "filename","final_verified_accuracy","problem_instances_count",
    "verified","falsified","timeout",
    "mean_time_all","max_time_all",
    "mean_time_safe","max_time_safe",
    "safe_incomplete_indices"
])

for filename in os.listdir(folder):
    if filename.endswith(".txt"):
        path = os.path.join(folder, filename)
        result = parse_verification_report(path)
        print(f"Results for {filename}:")
        print(result)
        results = pd.concat([results, pd.DataFrame([{"filename": filename, **result}])], ignore_index=True)
        print("-" * 40)

Results for data_index=73-76.txt:
{'final_verified_accuracy': 75.0, 'problem_instances_count': 4, 'verified': 3, 'falsified': 1, 'timeout': 0, 'mean_time_all': 0.6005850320838676, 'max_time_all': 1.2320044040679932, 'mean_time_safe': 0.6502602895100912, 'max_time_safe': 1.2320044040679932, 'safe_incomplete_indices': [1, 2, 3]}
----------------------------------------
Results for data_index=0-71.txt:
{'final_verified_accuracy': 97.22222222222221, 'problem_instances_count': 72, 'verified': 70, 'falsified': 2, 'timeout': 0, 'mean_time_all': 0.5151906059561167, 'max_time_all': 8.080828666687012, 'mean_time_safe': 0.5295758826392037, 'max_time_safe': 8.080828666687012, 'safe_incomplete_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71]}
------------------------------------

/tmp/ipykernel_1875/2815720875.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([{"filename": filename, **result}])], ignore_index=True)


In [4]:
results["total"] = results["verified"] + results["falsified"] + results["timeout"]

In [7]:
results["verified"].sum()

95

In [8]:
results["falsified"].sum()

5

In [6]:
results.head(20)

,filename,final_verified_accuracy,problem_instances_count,verified,falsified,timeout,mean_time_all,max_time_all,mean_time_safe,max_time_safe,safe_incomplete_indices,total
0,data_index=73-76.txt,75.000000,4,3,1,0,0.600585,1.232004,0.650260,1.232004,"[1, 2, 3]",4
1,data_index=0-71.txt,97.222222,72,70,2,0,0.515191,8.080829,0.529576,8.080829,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",72
2,data_index=79-80.txt,50.000000,2,1,1,0,0.501778,0.991666,0.991666,0.991666,[0],2
3,data_index=82-86.txt,100.000000,5,5,0,0,0.436388,0.998864,0.436389,0.998864,"[0, 1, 2, 3, 4]",5
4,data_index=89-91.txt,100.000000,3,3,0,0,0.557044,1.064052,0.557046,1.064052,"[0, 1, 2]",3
5,data_index=94.txt,100.000000,1,1,0,0,1.044091,1.044102,1.044102,1.044102,[0],1
6,data_index=96-97.txt,100.000000,2,2,0,0,0.676838,1.064196,0.676842,1.064196,"[0, 1]",2
7,data_index=100.txt,100.000000,1,1,0,0,1.212509,1.212522,1.212522,1.212522,[0],1
8,data_index=109.txt,100.000000,1,1,0,0,1.035812,1.035822,1.035822,1.035822,[0],1
9,data_index=117.txt,100.000000,1,1,0,0,1.050426,1.050437,1.050437,1.050437,[0],1


In [9]:
results["total_time"] = results["mean_time_all"] * results["total"]

In [10]:
results["total_time"].mean()

3.335106439330147